### Boston Crimes Database

The purpose of this notebook is to set up from scratch a local production PostgreSQL server using Boston Crimes data.

In [48]:
import psycopg2 as ps
import csv
import pandas as pd

#### Creating the database and the schema

In [16]:
def create_db(db, user, password, new_db):
    try:
        with ps.connect(database=db, user=user, password=password) as con:
            con.autocommit = True
            with con.cursor() as cur:    
                cur.execute(f"""CREATE DATABASE {new_db};""")
    finally:
        con.close()


In [17]:
create_db('postgres','postgres','','crime_db')

In [122]:
con = ps.connect(database = 'crime_db', user = 'postgres')

In [123]:
cur = con.cursor()

In [40]:
cur.execute("""CREATE SCHEMA crimes;""")

In [41]:
con.commit()

#### Obtaining the Column Names and Sample

In [47]:
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [68]:
df = pd.read_csv('boston.csv')
# tuple(df.iloc[:,2].tolist())

#### Creating a function for analyzing unique column values

In [70]:
def get_col_value_set(csv_filename, chunk, col_index):
    values = set()
    for chunk in pd.read_csv(csv_filename, chunksize = chunk):
        values.update(tuple(chunk.iloc[:,col_index].tolist()))
    
    return values    

In [73]:
for column in range(len(col_headers)):
    values = get_col_value_set('boston.csv',500,column)
    print(col_headers[column], len(values), sep = '\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


#### Getting max length of observation in description column

In [76]:
descriptions = get_col_value_set('boston.csv',500, 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
    
max_len

58

#### Creating the table

In [91]:
dic = {x:y for x,y in list(zip(col_headers,first_row))}
pd.DataFrame(dic, index=[0])

,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.35779134,-71.13937053


From what we see in the above dataframe we can safely designate incident_number and offense_code as integer datatypes.

Description column can be created as VARCHAR(100) to give a margin of error since the longest value in the column is about 58 characters.

Date would be a Date format and an enumerated datatype can be created for day_of_the_week column since there are only 7 values.

Lat and long need to maintain their precision and can be instantiated as decimal datatype.


In [136]:
cur.execute("""
    CREATE TYPE weekday_enum AS ENUM ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday');
    """)

cur.execute("""
    CREATE TABLE crimes.boston_crimes (
    incident_id INTEGER PRIMARY KEY,
    offense_id INTEGER,
    description VARCHAR(100),
    date DATE,
    day_of_the_week weekday_enum,
    lat DECIMAL,
    long DECIMAL
    );
    """)

In [137]:
con.commit()

#### Loading data into table

In [140]:
with open('boston.csv') as f:
    cur.copy_expert("""COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;""", f)

In [141]:
con.commit()

#### Revoking public privileges

In [142]:
cur.execute("""REVOKE ALL ON SCHEMA public FROM public;
                REVOKE ALL ON DATABASE crime_db FROM public; """)

#### Creating readonly group

In [144]:
cur.execute("""CREATE GROUP readonly NOLOGIN;""")
cur.execute("""GRANT CONNECT ON DATABASE crime_db TO readonly;""")
cur.execute("""GRANT USAGE ON SCHEMA crimes TO readonly;""")
cur.execute("""GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;""")

#### Creating readwrite group

In [145]:
cur.execute("""
    CREATE GROUP readwrite NOLOGIN;
    GRANT CONNECT ON DATABASE crime_db TO readwrite;
    GRANT USAGE ON SCHEMA crimes TO readwrite;
    GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;
    """)

#### Creating users for readonly and readwrite groups

In [146]:
cur.execute("""
    CREATE USER data_analyst WITH PASSWORD 'secret1';
    GRANT readonly TO data_analyst;
""")

cur.execute("""
    CREATE USER data_scientist WITH PASSWORD 'secret2';
    GRANT readwrite TO data_scientist;
""")

In [147]:
con.commit()

In [148]:
con.close()

#### Testing database

In [149]:
con = ps.connect(database = 'crime_db', user = 'postgres')
cur = con.cursor()

In [150]:
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
    """)

cur.fetchall()

[('readonly', False, False, False, False),
 ('readwrite', False, False, False, False),
 ('data_analyst', False, False, False, True),
 ('data_scientist', False, False, False, True)]

In [151]:
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")

cur.fetchall()

[('readonly', 'SELECT'),
 ('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [152]:
con.close()